# Release Presentation

In [1]:
%%capture
import sys
!{sys.executable} -m pip install google-auth-oauthlib
!{sys.executable} -m pip install google-api-python-client
!{sys.executable} -m pip install xhtml2pdf
!{sys.executable} -m pip install pdfrw

In [2]:
import requests
import pandas as pd
import re
from modules.presentations import *
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))

In [3]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

We get the json file with the information from
https://wmwaredata.s3.us-east-2.amazonaws.com/releases.json

In [4]:
url = 'https://wmwaredata.s3.us-east-2.amazonaws.com/releases.json'
r = requests.get(url, allow_redirects=True)
open('data/releases.json', 'wb').write(r.content)

286301

In [5]:
df0 = pd.read_json('data/releases.json')
df0

COLUMN_NAMES = ['assets', 'assets_url', 'author', 'body', 'created_at', 'draft', 'html_url', 
                'id', 'name', 'node_id', 'prerelease', 'published_at', 'tag_name', 
                'tarball_url', 'target_commitish', 'upload_url', 'url', 'zipball_url']
df = pd.DataFrame(columns=COLUMN_NAMES)

for i in range(len(df0)):
    for j in range(len(df0.columns)):
        my_dict = df0.iloc[i][j]
        if my_dict is not None:
            df = pd.concat([df, pd.DataFrame([my_dict])])
df = df.reset_index(drop=True)

repos = []
for i in range(len(df)):
    s = df['assets_url'].iloc[i]
    repo = re.search('https://api.github.com/repos/k8-proxy/(.*)/releases/', s)
    repo = repo.group(1)
    repos.append(repo)
df["repo"] = repos

# Reorder columns
cols = list(df.columns)
cols = [cols[-1]] + cols[0:-1]
df = df[cols]

# select only the desired features
df = df[['repo', 'html_url', 'tag_name', 'id', 'created_at']]
df

,repo,html_url,tag_name,id,created_at
0,aws-jmeter-test-engine,https://github.com/k8-proxy/aws-jmeter-test-en...,v0.2.1,37081347,2021-01-29T11:22:14Z
1,aws-jmeter-test-engine,https://github.com/k8-proxy/aws-jmeter-test-en...,v0.2.0,36769277,2021-01-22T12:04:31Z
2,aws-jmeter-test-engine,https://github.com/k8-proxy/aws-jmeter-test-en...,v0.1.0,32809551,2020-10-20T09:00:02Z
3,glasswall-desktop,https://github.com/k8-proxy/glasswall-desktop/...,v1.0.0,35488083,2020-12-18T17:24:35Z
4,glasswall-desktop,https://github.com/k8-proxy/glasswall-desktop/...,v0.9.0,34501707,2020-11-27T06:53:49Z
5,glasswall-desktop,https://github.com/k8-proxy/glasswall-desktop/...,v0.1.7,34263935,2020-11-21T11:35:04Z
6,glasswall-desktop,https://github.com/k8-proxy/glasswall-desktop/...,v0.1.6,33936626,2020-11-13T19:39:27Z
7,glasswall-desktop,https://github.com/k8-proxy/glasswall-desktop/...,v0.1.5,33573630,2020-11-06T16:07:08Z
8,glasswall-desktop,https://github.com/k8-proxy/glasswall-desktop/...,v0.1.4,33227072,2020-10-29T15:35:44Z
9,glasswall-desktop,https://github.com/k8-proxy/glasswall-desktop/...,v0.1.3,32864364,2020-10-21T12:57:57Z


In [6]:
presentation_id, slides_ids = create_presentation(df, file_name='Release Presentation', table_title='Releases', step = 8)

The presentation is in this folder: https://drive.google.com/drive/u/1/folders/1dELfGV6IMMII97tTjqSXeJwPAMEjtbDX